# DA-01: Diseño de Modelo Dimensional para Analítica de Inventarios

## 📋 Contexto del Caso de Negocio

**Empresa:** Cadena de retail nacional "SuperRetail" con 30 centros de distribución y 200 tiendas.

**Situación:** La empresa tiene datos de inventario dispersos en múltiples sistemas operacionales (WMS, ERP, TMS). Los analistas necesitan responder preguntas como:
- ¿Cuál es la rotación de inventario por categoría y ubicación?
- ¿Qué productos tienen mayor cobertura (días de inventario)?
- ¿Dónde están los inventarios obsoletos o de lento movimiento?
- ¿Cómo evoluciona el valor del inventario mes a mes?

**Objetivo:** Diseñar e implementar un modelo dimensional (esquema estrella) que permita análisis OLAP rápidos y flexibles.

---

## 🎯 Qué - Por qué - Para qué - Cuándo - Cómo

### ❓ ¿QUÉ estamos haciendo?
Diseñando un **modelo dimensional tipo estrella** con:
- **Tabla de hechos:** `fact_inventory_daily` con métricas diarias de inventario
- **Dimensiones:** `dim_date`, `dim_product`, `dim_location`, `dim_product_category`
- **Métricas calculadas:** rotación, cobertura, valor inventario, unidades disponibles

### 🔍 ¿POR QUÉ es importante?
- **Velocidad:** El esquema estrella optimiza queries analíticas (menos JOINs)
- **Simplicidad:** Los analistas de negocio pueden navegar intuitivamente
- **Historización:** Las dimensiones tipo SCD (Slowly Changing Dimension) preservan historia
- **Agregaciones:** Facilita cubos OLAP y reportes drill-down

### 🎁 ¿PARA QUÉ sirve?
- Dashboards ejecutivos de inventario (BI)
- Análisis ABC/XYZ de productos
- Detección de obsolescencia y slow-movers
- Input para modelos de optimización de inventario
- Auditorías y reconciliaciones contables

### ⏰ ¿CUÁNDO aplicarlo?
- **Cuando hay múltiples fuentes** de datos de inventario que necesitan consolidarse
- **Cuando los queries son lentos** en el modelo transaccional normalizado
- **Cuando necesitas análisis históricos** con comparaciones temporales
- **Cuando diferentes áreas** (finanzas, operaciones, comercial) consultan los mismos datos

### 🛠️ ¿CÓMO lo hacemos?
1. **Análisis de requerimientos:** Identificar preguntas de negocio y KPIs
2. **Definir granularidad:** SKU x Ubicación x Día
3. **Identificar dimensiones:** Tiempo, Producto, Ubicación, Categoría
4. **Definir métricas:** Qty on-hand, valor, entradas, salidas, rotación
5. **Diseñar claves:** Surrogate keys (SK) vs natural keys
6. **Implementar ETL:** Cargar dimensiones y hechos desde sources
7. **Validar:** Queries de prueba y reconciliación con fuentes

In [ ]:
# Imports necesarios
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuración visual
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Librerías cargadas")

## 📥 Paso 1: Cargar datos transaccionales (fuentes)

Simulamos la carga desde sistemas operacionales (WMS, ERP).

In [ ]:
# Rutas de datos
data_path = Path('../../data/raw')

# Cargar datasets sintéticos
df_inventory = pd.read_csv(data_path / 'inventory.csv', parse_dates=['date'])
df_products = pd.read_csv(data_path / 'products.csv')
df_locations = pd.read_csv(data_path / 'locations.csv')

print(f"📦 Inventory records: {len(df_inventory):,}")
print(f"📦 Products: {len(df_products):,}")
print(f"📍 Locations: {len(df_locations):,}")

# Vista preliminar
print("\n🔍 Sample inventory data:")
df_inventory.head(3)

## 🏗️ Paso 2: Construir Dimensión Tiempo (dim_date)

La dimensión tiempo es fundamental en modelos dimensionales. Incluye atributos calendáricos para facilitar análisis temporales.

In [ ]:
# Crear dimensión de tiempo
def build_date_dimension(start_date, end_date):
    """
    Genera tabla de dimensión de fechas con atributos calendáricos.
    """
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    dim_date = pd.DataFrame({
        'date_key': date_range.strftime('%Y%m%d').astype(int),  # Surrogate key
        'date': date_range,
        'year': date_range.year,
        'quarter': date_range.quarter,
        'month': date_range.month,
        'month_name': date_range.strftime('%B'),
        'week': date_range.isocalendar().week,
        'day_of_month': date_range.day,
        'day_of_week': date_range.dayofweek,
        'day_name': date_range.strftime('%A'),
        'is_weekend': date_range.dayofweek.isin([5, 6]).astype(int),
        'is_month_start': date_range.is_month_start.astype(int),
        'is_month_end': date_range.is_month_end.astype(int)
    })
    
    return dim_date

# Construir dimensión
min_date = df_inventory['date'].min()
max_date = df_inventory['date'].max()

dim_date = build_date_dimension(min_date, max_date)

print(f"📅 Dimensión tiempo creada: {len(dim_date)} días")
print(f"📅 Rango: {min_date.date()} a {max_date.date()}")
print("\n🔍 Sample:")
dim_date.head()

## 🏷️ Paso 3: Construir Dimensión Producto (dim_product)

Incluye jerarquías de categorización para drill-down/roll-up.

In [ ]:
# Dimensión producto con jerarquía
dim_product = df_products.copy()
dim_product['product_key'] = range(1, len(dim_product) + 1)  # Surrogate key

# Reordenar columnas con SK primero
dim_product = dim_product[[
    'product_key', 'sku', 'product_name', 'category', 'subcategory',
    'brand', 'unit_cost', 'unit_price', 'weight_kg', 'is_active'
]]

print(f"🏷️ Dimensión producto creada: {len(dim_product)} SKUs")
print(f"📊 Categorías: {dim_product['category'].nunique()}")
print(f"📊 Subcategorías: {dim_product['subcategory'].nunique()}")
print("\n🔍 Sample:")
dim_product.head(3)

## 📍 Paso 4: Construir Dimensión Ubicación (dim_location)

Incluye jerarquía geográfica: región > ciudad > ubicación.

In [ ]:
# Dimensión ubicación
dim_location = df_locations.copy()
dim_location['location_key'] = range(1, len(dim_location) + 1)  # Surrogate key

# Reordenar columnas
dim_location = dim_location[[
    'location_key', 'location_id', 'location_name', 'location_type',
    'region', 'city', 'capacity_units', 'is_active'
]]

print(f"📍 Dimensión ubicación creada: {len(dim_location)} ubicaciones")
print(f"📊 Tipos: {dim_location['location_type'].value_counts().to_dict()}")
print(f"📊 Regiones: {dim_location['region'].nunique()}")
print("\n🔍 Sample:")
dim_location.head(3)

## 📊 Paso 5: Construir Tabla de Hechos (fact_inventory_daily)

La tabla de hechos contiene métricas numéricas a la granularidad definida: **SKU x Ubicación x Día**.

In [ ]:
# Crear tabla de hechos
fact_inventory = df_inventory.copy()

# Join con dimensiones para obtener surrogate keys
fact_inventory = fact_inventory.merge(
    dim_date[['date', 'date_key']], on='date', how='left'
)

fact_inventory = fact_inventory.merge(
    dim_product[['sku', 'product_key']], on='sku', how='left'
)

fact_inventory = fact_inventory.merge(
    dim_location[['location_id', 'location_key']], on='location_id', how='left'
)

# Agregar métricas calculadas
fact_inventory = fact_inventory.merge(
    dim_product[['product_key', 'unit_cost']], on='product_key', how='left'
)

fact_inventory['inventory_value'] = fact_inventory['qty_on_hand'] * fact_inventory['unit_cost']

# Seleccionar columnas finales de la tabla de hechos
fact_inventory_final = fact_inventory[[
    'date_key', 'product_key', 'location_key',
    'qty_on_hand', 'qty_reserved', 'qty_available',
    'inventory_value'
]]

print(f"📊 Tabla de hechos creada: {len(fact_inventory_final):,} registros")
print(f"💰 Valor total inventario: ${fact_inventory_final['inventory_value'].sum():,.2f}")
print(f"📦 Total unidades: {fact_inventory_final['qty_on_hand'].sum():,.0f}")
print("\n🔍 Sample:")
fact_inventory_final.head()

## 🔍 Paso 6: Queries Analíticas - Ejemplos de Uso

Demostramos cómo el modelo dimensional facilita consultas de negocio complejas.

In [ ]:
# Query 1: Inventario por categoría y región
query1 = fact_inventory_final.merge(
    dim_product[['product_key', 'category']], on='product_key'
).merge(
    dim_location[['location_key', 'region']], on='location_key'
).groupby(['category', 'region']).agg({
    'qty_on_hand': 'sum',
    'inventory_value': 'sum'
}).round(2)

print("📊 Query 1: Inventario por Categoría y Región")
print(query1.head(10))

In [ ]:
# Query 2: Top 10 SKUs por valor de inventario
query2 = fact_inventory_final.merge(
    dim_product[['product_key', 'sku', 'product_name', 'category']], on='product_key'
).groupby(['sku', 'product_name', 'category']).agg({
    'inventory_value': 'sum'
}).sort_values('inventory_value', ascending=False).head(10)

print("\n💰 Query 2: Top 10 SKUs por Valor de Inventario")
print(query2.round(2))

In [ ]:
# Query 3: Evolución temporal del inventario por mes
fact_with_time = fact_inventory_final.merge(
    dim_date[['date_key', 'year', 'month', 'month_name']], on='date_key'
)

query3 = fact_with_time.groupby(['year', 'month', 'month_name']).agg({
    'qty_on_hand': 'sum',
    'inventory_value': 'sum'
}).reset_index().sort_values(['year', 'month'])

print("\n📈 Query 3: Evolución Mensual del Inventario")
print(query3.round(2))

## 📊 Paso 7: Visualizaciones del Modelo Dimensional

In [ ]:
# Visualización 1: Valor de inventario por categoría
viz1_data = fact_inventory_final.merge(
    dim_product[['product_key', 'category']], on='product_key'
).groupby('category')['inventory_value'].sum().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(12, 6))
viz1_data.plot(kind='bar', ax=ax, color='steelblue')
ax.set_title('Valor de Inventario por Categoría', fontsize=14, fontweight='bold')
ax.set_xlabel('Categoría')
ax.set_ylabel('Valor ($)')
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print(f"💡 Insight: La categoría '{viz1_data.index[0]}' concentra ${viz1_data.values[0]:,.0f} ({viz1_data.values[0]/viz1_data.sum()*100:.1f}%) del valor total")

In [ ]:
# Visualización 2: Distribución de inventario por tipo de ubicación
viz2_data = fact_inventory_final.merge(
    dim_location[['location_key', 'location_type']], on='location_key'
).groupby('location_type')['inventory_value'].sum()

fig, ax = plt.subplots(figsize=(10, 6))
viz2_data.plot(kind='pie', ax=ax, autopct='%1.1f%%', startangle=90)
ax.set_title('Distribución de Valor de Inventario por Tipo de Ubicación', fontsize=14, fontweight='bold')
ax.set_ylabel('')
plt.tight_layout()
plt.show()

## 💾 Paso 8: Exportar Modelo Dimensional

Guardamos las tablas dimensionales en formato Parquet para uso posterior.

In [ ]:
# Crear directorio de salida
output_path = Path('../../data/processed/dimensional_model')
output_path.mkdir(parents=True, exist_ok=True)

# Exportar dimensiones y hechos
dim_date.to_parquet(output_path / 'dim_date.parquet', index=False)
dim_product.to_parquet(output_path / 'dim_product.parquet', index=False)
dim_location.to_parquet(output_path / 'dim_location.parquet', index=False)
fact_inventory_final.to_parquet(output_path / 'fact_inventory_daily.parquet', index=False)

print("✅ Modelo dimensional exportado exitosamente")
print(f"📁 Ubicación: {output_path.absolute()}")
print("\n📦 Archivos generados:")
for file in output_path.glob('*.parquet'):
    size_mb = file.stat().st_size / (1024 * 1024)
    print(f"  - {file.name}: {size_mb:.2f} MB")

## 📚 Resumen y Conclusiones

### ✅ Lo que logramos:
1. ✅ **Modelo dimensional tipo estrella** con 3 dimensiones + 1 tabla de hechos
2. ✅ **Surrogate keys** para independencia de sistemas source
3. ✅ **Jerarquías navegables:** Categoría → Subcategoría → SKU | Región → Ciudad → Ubicación
4. ✅ **Métricas precalculadas:** Valor de inventario para performance
5. ✅ **Queries optimizadas:** Análisis multidimensionales con JOINs simples

### 🎯 Aplicaciones prácticas:
- **Dashboards BI:** Conexión directa a Power BI, Tableau, Looker
- **Cubos OLAP:** Pre-agregaciones para queries subsegundo
- **Machine Learning:** Features estructuradas para modelos predictivos
- **Auditorías:** Trazabilidad histórica de inventarios

### 🚀 Próximos pasos:
1. Implementar **SCD Type 2** en dim_product para rastrear cambios de precio/categoría
2. Agregar dimensión `dim_inventory_status` (disponible, reservado, dañado)
3. Crear **agregados** mensuales para optimizar queries de largo plazo
4. Implementar **particionamiento** por fecha en la tabla de hechos
5. Añadir métricas calculadas: rotación, cobertura, obsolescencia

### 📖 Referencias:
- Kimball, R. & Ross, M. (2013). *The Data Warehouse Toolkit*
- Inmon, W.H. (2005). *Building the Data Warehouse*